In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 1.74 ms (started: 2021-01-13 12:54:42 +00:00)


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
time: 26.6 s (started: 2021-01-13 12:54:42 +00:00)


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

time: 1.97 ms (started: 2021-01-13 12:55:09 +00:00)


In [4]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle
time: 8.4 ms (started: 2021-01-13 12:55:09 +00:00)


In [5]:
#!kaggle datasets download -d nih-chest-xrays/data
#!ls

time: 752 µs (started: 2021-01-13 12:55:09 +00:00)


In [6]:
import pandas as pd
df = pd.read_csv('Data_Entry_2017.csv')
df = df[df['Image Index'] < '00006501_000.png']
df = df[df['Finding Labels'].apply(lambda x: '|' not in x)]
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df = df.iloc[:3000,:2]
df

,Image Index,Finding Labels
0,00000001_000.png,Cardiomegaly
1,00000002_000.png,No Finding
2,00000003_000.png,Hernia
3,00000003_001.png,Hernia
4,00000003_002.png,Hernia
...,...,...
2995,00000980_001.png,No Finding
2996,00000980_002.png,No Finding
2997,00000980_003.png,Infiltration
2998,00000980_004.png,No Finding


time: 642 ms (started: 2021-01-13 12:55:09 +00:00)


In [7]:
MAIN_DIR = '/content/gdrive/My Drive/Kaggle'
temp = 'images/images main/'
path = os.path.join(MAIN_DIR,temp)
path

'/content/gdrive/My Drive/Kaggle/images/images main/'

time: 3.86 ms (started: 2021-01-13 12:55:09 +00:00)


In [8]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np

target = ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema',"Emphysema",'Fibrosis',
          'Effusion',"Pneumonia","Pleural_thickening",'Cardiomegaly','Nodule', 'Mass','Hernia','No Finding']
images = []
label = []
flat_data = []
for i in target:
  for j in df[df['Finding Labels'] == i]['Image Index']:
    for img in os.listdir(path):
      if img == j:
        arr = imread(os.path.join(path,img))
        #print(arr.shape)
        arr_res = resize(arr,(224,224,3))
        #print(arr_res.shape)
        flat_data.append(arr_res)
        images.append(img)
        label.append(target.index(i))

flat_data = np.array(flat_data)
images = np.array(images)
label = np.array(label)

time: 21min 14s (started: 2021-01-13 12:55:09 +00:00)


In [9]:
# images

time: 1.04 ms (started: 2021-01-13 13:16:24 +00:00)


In [10]:
# len(label)

time: 1.71 ms (started: 2021-01-13 13:16:24 +00:00)


In [11]:
flat_data.shape

(2955, 224, 224, 3)

time: 3.56 ms (started: 2021-01-13 13:16:24 +00:00)


In [12]:
np.unique(label,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14]),
 array([ 143,   44,  296,   63,    9,   42,   74,  126,   12,   63,   78,
          42,   16, 1947]))

time: 4.32 ms (started: 2021-01-13 13:16:24 +00:00)


In [13]:
# from keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2
#                                    )

# test_datagen = ImageDataGenerator(rescale = 1./255)

time: 1.69 ms (started: 2021-01-13 13:16:24 +00:00)


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(flat_data,label,test_size = 0.3)

time: 3.64 s (started: 2021-01-13 13:16:24 +00:00)


In [15]:
X_train.shape

(2068, 224, 224, 3)

time: 2.8 ms (started: 2021-01-13 13:16:27 +00:00)


In [16]:
y_train[0]

14

time: 2.63 ms (started: 2021-01-13 13:16:27 +00:00)


In [17]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=15)
y_test = to_categorical(y_test,num_classes=15)
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
      dtype=float32)

time: 1.87 s (started: 2021-01-13 13:16:27 +00:00)


In [18]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
incep = InceptionV3(input_shape=[224,224,3],weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step
time: 3.7 s (started: 2021-01-13 13:16:29 +00:00)


In [19]:
for layer in incep.layers:
  layer.trainable = False

time: 13.7 ms (started: 2021-01-13 13:16:33 +00:00)


In [20]:
# base_model.summary()

time: 800 µs (started: 2021-01-13 13:16:33 +00:00)


In [21]:
from keras.models import Model
from tensorflow.keras import layers
from keras.layers import Input, Lambda, Dense, Flatten
# input_shape=(40000,)
# img_input = Input(shape=input_shape)
x = Flatten()(incep.output)
pre = Dense(15, activation='softmax')(x)
model = Model(inputs=incep.input,outputs=pre)

time: 101 ms (started: 2021-01-13 13:16:33 +00:00)


In [22]:
# model.summary()

time: 795 µs (started: 2021-01-13 13:16:33 +00:00)


In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy')

time: 31.3 ms (started: 2021-01-13 13:16:33 +00:00)


In [25]:
model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
65/65 [==============================] - 265s 4s/step - loss: 8.1794 - val_loss: 4.5151
Epoch 2/10
65/65 [==============================] - 261s 4s/step - loss: 1.8239 - val_loss: 4.6224
Epoch 3/10
65/65 [==============================] - 262s 4s/step - loss: 1.0449 - val_loss: 4.2431
Epoch 4/10
65/65 [==============================] - 261s 4s/step - loss: 0.5287 - val_loss: 6.1686
Epoch 5/10
65/65 [==============================] - 261s 4s/step - loss: 0.5382 - val_loss: 4.8549
Epoch 6/10
65/65 [==============================] - 262s 4s/step - loss: 0.2660 - val_loss: 4.9420
Epoch 7/10
65/65 [==============================] - 261s 4s/step - loss: 0.4719 - val_loss: 5.2274
Epoch 8/10
65/65 [==============================] - 260s 4s/step - loss: 0.5336 - val_loss: 5.2395
Epoch 9/10
65/65 [==============================] - 262s 4s/step - loss: 0.1692 - val_loss: 5.6032
Epoch 10/10
65/65 [==============================] - 261s 4s/step - loss: 0.5611 - val_loss: 6.5421


time: 43min 37s (started: 2021-01-13 13:17:30 +00:00)


In [26]:
model.save('chestx_ray.h5')

time: 2.11 s (started: 2021-01-13 14:02:02 +00:00)


In [27]:
!pip install streamlit &> /dev/null
!pip install pyngrok==4.1.1 &> /dev/null

time: 20.1 s (started: 2021-01-13 14:02:06 +00:00)


In [28]:
%%writefile Image_Classifier.py
import streamlit as st
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
import tensorflow as tf
import numpy as np
st.title("Chest X_ray Classifier")
st.write("Upload a chest x_ray image and the model will predict the disease that person can have!")
upload = st.file_uploader("Choose an Image")
model = tf.keras.models.load_model('chestx_ray.h5')
target = ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema',"Emphysema",'Fibrosis',
          'Effusion',"Pneumonia","Pleural_thickening",'Cardiomegaly','Nodule', 'Mass','Hernia','No Finding']
if upload is not None:
  img = Image.open(upload)
  st.image(img,caption = 'UPLOADED IMAGE',use_column_width=True)

  if st.button('DETECT'):
    li=[]
    img = np.array(img)
    ar1 = resize(img,(224,224,3))
    li.append(ar1)
    li = np.array(li)
    y1 = model.predict(li)
    st.write("Disease is:")
    st.title(target[y1.argmax()])

Overwriting Image_Classifier.py
time: 234 ms (started: 2021-01-13 14:02:32 +00:00)


In [29]:
from pyngrok import ngrok
!nohup streamlit run Image_Classifier.py &
r_url = ngrok.connect(port='8501')
print(r_url)

nohup: appending output to 'nohup.out'
http://d8419880c142.ngrok.io
time: 1.67 s (started: 2021-01-13 14:02:37 +00:00)
